<a href="https://colab.research.google.com/github/iankim20/-/blob/AI-and-medicine/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive                #구글드라이브에 마운트
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

LITS_DIR = '/gdrive/Team Drives/SevAIClass/LITS17'
# TRAIN_DIR = '/gdrive/Team Drives/SevAIClass/'


In [ ]:
# 1부터 130까지 랜덤으로 100, 15, 15로 구획하기

train_pt = np.sort(np.random.choice(131, 100, replace=False))

y=range(131)
validtest = np.delete(y,train_pt)
valid_pt = np.sort(np.random.choice(validtest, 15, replace=False))            #np.delete는 주어진 index만큼 원래 array에서 지워주는 함수
test_pt = np.sort(np.delete(y, np.append(train_pt, valid_pt)))

print(list(train_pt))
print(list(valid_pt))
print(list(test_pt))


[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 15, 16, 18, 19, 21, 22, 23, 24, 25, 26, 28, 29, 32, 33, 35, 36, 38, 39, 40, 41, 42, 43, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 72, 73, 75, 76, 77, 79, 80, 81, 82, 85, 86, 87, 89, 90, 92, 95, 96, 97, 98, 99, 100, 102, 103, 105, 106, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 121, 122, 123, 125, 126, 127, 128, 130]
[13, 14, 17, 20, 31, 34, 44, 71, 74, 78, 94, 101, 107, 124, 129]
[9, 27, 30, 37, 51, 57, 58, 68, 83, 84, 88, 91, 93, 104, 108, 117]


In [ ]:
train_pt=[0, 1, 2, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 28, 30, 31, 33, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 59, 62, 63, 66, 67, 70, 71, 72, 73, 74, 75, 76, 77, 79, 81, 82, 83, 84, 85, 86, 88, 89, 90, 91, 93, 94, 95, 96, 97, 98, 100, 101, 103, 104, 105, 106, 107, 108, 111, 112, 113, 114, 115, 117, 119, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130]
train_pt = np.array(train_pt)

valid_pt = np.array([13, 14, 17, 20, 31, 34, 44, 71, 74, 78, 94, 101, 107, 124, 129])
test_pt = np.array([9, 27, 30, 37, 51, 57, 58, 68, 83, 84, 88, 91, 93, 104, 108, 117])


np.shape(test_pt)

(16,)

In [ ]:
from tqdm import tqdm


img= nib.load(os.path.join(LITS_DIR, 'volume-%d.nii' % 30))
data=img.get_fdata()                                                 #data shape는 3번째 값이 i마다 다를 수 있지만 512,512, 121 형태


for i in tqdm((40,41,42)):
  fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/pr-"+str(i)+".npy"
  tmp_arr= data[:, :, i]
  
  print(fname_template)
  print(tmp_arr.shape)
  print(tmp_arr.dtype)
  
  np.save(fname_template, tmp_arr.astype(np.int16)) # x_save.npy
  
  
'''tmp_arr=np.delete(tmp_arr, 0,0)
print(np.shape(tmp_arr))
tmp_arr'''



100%|██████████| 3/3 [00:00<00:00, 90.32it/s]

/gdrive/Team Drives/SevAIClass/Working/KYT/pr-40.npy
(512, 512)
float64
/gdrive/Team Drives/SevAIClass/Working/KYT/pr-41.npy
(512, 512)
float64
/gdrive/Team Drives/SevAIClass/Working/KYT/pr-42.npy
(512, 512)
float64


'tmp_arr=np.delete(tmp_arr, 0,0)\nprint(np.shape(tmp_arr))\ntmp_arr'

In [ ]:
!ls


sample_data


In [ ]:
# liver와 no_liver 개수 세기
from tqdm import tqdm              #progress bar 설치

liver=0
no_liver=0

for i in tqdm(train_pt):
  img= nib.load(os.path.join(LITS_DIR, 'segmentation-%d.nii' % i))
  data=img.get_fdata()                                                 #data shape는 3번째 값이 i마다 다를 수 있지만 512,512, 121 형태
  z_num=data.shape[2]
  tdata=np.transpose(data,(1,0,2))                                     #x,y 바꿔줌 -> tdata
  for j in range(z_num):
    if np.count_nonzero(tdata[:, :, j])==0:
      no_liver+=1
      
    else:
      liver+=1
    
print(liver, no_liver)
      
  


14649 29861


In [ ]:
#train 저장하기_각각 liver와 non_liver로

from tqdm import tqdm              #progress bar 설치
from PIL import Image

num_liver=0
num_no_liver=0

for i in tqdm(train_pt):
  img_vol= nib.load(os.path.join(LITS_DIR, 'volume-%d.nii' % i))
  img_seg= nib.load(os.path.join(LITS_DIR, 'segmentation-%d.nii' % i))     #liver 또는 non_liver로 나누기 위해 처음이자 마지막으로 seg로 받음
  data=img_vol.get_fdata()                                                 #data shape는 3번째 값이 i마다 다를 수 있지만 512,512, 121 형태
  label=img_seg.get_fdata()
  
  z_num=data.shape[2]
  tdata=np.transpose(data,(1,0,2))                                     #x,y 바꿔줌 -> tdata
  #여기서부터 tdata[:, :, j]를 -175부터 274로 짜르고, 크기를 512를 256로 줄여주자
  tdata = np.clip(tdata, -175, 275)
  tdata = np.interp(tdata, (-175, 275), (0,255))
  
  for j in range(z_num):
    if np.count_nonzero(label[:, :, j])==0:                           #liver가 없다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Train_set/non_liver/train_non_liver-"+str(num_no_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")                                        #convert("L")은 grey scale임을 알려주는 것
      pil_im.save(fname_template, "PNG")                                  # x_save.png
      
      num_no_liver += 1
      
    else:                                                             #liver가 있다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Train_set/liver/train_liver-"+str(num_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")
      pil_im.save(fname_template, "PNG")
      
      num_liver+=1
    


100%|██████████| 100/100 [51:43<00:00, 39.86s/it]


In [ ]:
import os

path="/gdrive/Team Drives/SevAIClass/Working/KYT/Valid_set/"

try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
else:  
    print ("Successfully created the directory %s " % path)

Successfully created the directory /gdrive/Team Drives/SevAIClass/Working/KYT/Valid_set/ 


In [ ]:
# valid set과 test set을 각각 liver와 nonliver로 나눠서 저장하기-test는 분류후 나중에 알고리즘이 얼만큼 맞추는지 보는거로 판단

from tqdm import tqdm              #progress bar 설치
from PIL import Image

num_liver=0
num_no_liver=0

for i in tqdm(valid_pt):
  img_vol= nib.load(os.path.join(LITS_DIR, 'volume-%d.nii' % i))
  img_seg= nib.load(os.path.join(LITS_DIR, 'segmentation-%d.nii' % i))     #liver 또는 non_liver로 나누기 위해 처음이자 마지막으로 seg로 받음
  data=img_vol.get_fdata()                                                 #data shape는 3번째 값이 i마다 다를 수 있지만 512,512, 121 형태
  label=img_seg.get_fdata()
  
  z_num=data.shape[2]
  tdata=np.transpose(data,(1,0,2))                                     #x,y 바꿔줌 -> tdata
  #여기서부터 tdata[:, :, j]를 -175부터 274로 짜르고, 크기를 512를 256로 줄여주자
  tdata = np.clip(tdata, -175, 275)
  tdata = np.interp(tdata, (-175, 275), (0,255))
  
  for j in range(z_num):
    if np.count_nonzero(label[:, :, j])==0:                           #liver가 없다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Valid_set/non_liver/valid_non_liver-"+str(num_no_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")                                        #convert("L")은 grey scale임을 알려주는 것
      pil_im.save(fname_template, "PNG")                                  # x_save.png
      
      num_no_liver += 1
      
    else:                                                             #liver가 있다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Valid_set/liver/valid_liver-"+str(num_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")
      pil_im.save(fname_template, "PNG")
      
      num_liver+=1
    
num_liver=0
num_no_liver=0

for i in tqdm(test_pt):
  img_vol= nib.load(os.path.join(LITS_DIR, 'volume-%d.nii' % i))
  img_seg= nib.load(os.path.join(LITS_DIR, 'segmentation-%d.nii' % i))     #liver 또는 non_liver로 나누기 위해 처음이자 마지막으로 seg로 받음
  data=img_vol.get_fdata()                                                 #data shape는 3번째 값이 i마다 다를 수 있지만 512,512, 121 형태
  label=img_seg.get_fdata()
  
  z_num=data.shape[2]
  tdata=np.transpose(data,(1,0,2))                                     #x,y 바꿔줌 -> tdata
  #여기서부터 tdata[:, :, j]를 -175부터 274로 짜르고, 크기를 512를 256로 줄여주자
  tdata = np.clip(tdata, -175, 275)
  tdata = np.interp(tdata, (-175, 275), (0,255))
  
  for j in range(z_num):
    if np.count_nonzero(label[:, :, j])==0:                           #liver가 없다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Test_set/non_liver/test_non_liver-"+str(num_no_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")                                        #convert("L")은 grey scale임을 알려주는 것
      pil_im.save(fname_template, "PNG")                                  # x_save.png
      
      num_no_liver += 1
      
    else:                                                             #liver가 있다
      fname_template = "/gdrive/Team Drives/SevAIClass/Working/KYT/Test_set/liver/test_liver-"+str(num_liver)+".png"
      pil_im = Image.fromarray (tdata[:, :, j].astype(np.uint8))
      pil_im = pil_im.convert("L")
      pil_im.save(fname_template, "PNG")
      
      num_liver+=1




  0%|          | 0/15 [00:00<?, ?it/s]

FileNotFoundError: ignored

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms



transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

KeyboardInterrupt: ignored

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)              #여기에 Resnet 넣으면 ResNet 되는거임. 여기만 고치면됨
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

      
# Device setting          새로 추가함!!
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(dev)

net = Net().to(dev)         #여기서도 to(dev) 추가함!



In [ ]:
import torch.optim as optim
​
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(dev), labels.to(dev)         #여기도 새로 추가함!!

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [ ]:
!nvidia-smi

Wed Jun 26 06:19:08 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  